In [50]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
warnings.filterwarnings('ignore')

In [119]:
df

,abstract,articleTitle,Journaltitle,volume,pubDate,authors,MeSh,citations,affiliations,keywords,IsReviewArticle
PMID,,,,,,,,,,,
35961771,Navigation through complex environments requir...,Coordination between eye movement and whisking...,eNeuro,1,2022-Aug-12,NaN,None,0,[],None,0
35961623,Recent evidence shows that genetic and environ...,Polygenic risk for schizophrenia as a moderato...,Progress in neuro-psychopharmacology & biologi...,1,2022-Aug-09,NaN,None,0,[],"Adversity, Bipolar disorder, Genetics, Psychos...",0
35961621,This study aimed to examine the prospective as...,The association of substance use with attainin...,Preventive medicine,1,2022-Aug-09,NaN,None,0,[],"Alcohol use, Cannabis use, Employment, Tobacco...",0
35961602,Psychosis presentation can be affected by gene...,Differences of affective and non-affective psy...,Journal of affective disorders,1,2022-Aug-09,NaN,None,0,[],"Bipolar disorder, Depression, First-episode, P...",0
35961582,Mental health and cognitive achievement are pa...,Brain structure and function show distinct rel...,Biological psychiatry. Cognitive neuroscience ...,1,2022-Aug-09,NaN,None,0,[],"cognition, genetics, mental health, polygenic ...",0
...,...,...,...,...,...,...,...,...,...,...,...
32848624,Hearing loss is the third most common chronic ...,Berbamine Analogs Exhibit Differential Protect...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,58,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","aminoglycoside, berbamine, hair cell, hearing ...",0
32848620,Tyrosine hydroxylase (Th) expression has previ...,Purkinje Cell-Specific Knockout of Tyrosine Hy...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,124,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","catecholamine, cerebellum, cognition, dopamine...",0
32848619,Myelin is a dynamic membrane that is important...,That's a Wrap! Molecular Drivers Governing Neu...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,48,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","Caspr, Nogo-A, PrPc, myelin, nogo receptor, pa...",1


In [142]:
data  = pd.read_csv('../raw_data/neuroscience_data_100000.csv').drop(columns=['Unnamed: 0']).set_index('PMID').dropna(subset=['abstract','articleTitle'])
df_clean_title = data[~data.articleTitle.str.contains("{")]
df = df_clean_title[~df_clean_title.abstract.str.contains("{")]
df = df[df.abstract != '.']
df


,abstract,articleTitle,Journaltitle,volume,pubDate,authors,MeSh,citations,affiliations,keywords,IsReviewArticle
PMID,,,,,,,,,,,
35961771,Navigation through complex environments requir...,Coordination between eye movement and whisking...,eNeuro,1,2022-Aug-12,NaN,None,0,[],None,0
35961623,Recent evidence shows that genetic and environ...,Polygenic risk for schizophrenia as a moderato...,Progress in neuro-psychopharmacology & biologi...,1,2022-Aug-09,NaN,None,0,[],"Adversity, Bipolar disorder, Genetics, Psychos...",0
35961621,This study aimed to examine the prospective as...,The association of substance use with attainin...,Preventive medicine,1,2022-Aug-09,NaN,None,0,[],"Alcohol use, Cannabis use, Employment, Tobacco...",0
35961602,Psychosis presentation can be affected by gene...,Differences of affective and non-affective psy...,Journal of affective disorders,1,2022-Aug-09,NaN,None,0,[],"Bipolar disorder, Depression, First-episode, P...",0
35961582,Mental health and cognitive achievement are pa...,Brain structure and function show distinct rel...,Biological psychiatry. Cognitive neuroscience ...,1,2022-Aug-09,NaN,None,0,[],"cognition, genetics, mental health, polygenic ...",0
...,...,...,...,...,...,...,...,...,...,...,...
32848625,The inferior colliculus (IC) is an auditory mi...,Developmentally Regulated Rebound Depolarizati...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,74,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","action potential, auditory system, brain slice...",0
32848624,Hearing loss is the third most common chronic ...,Berbamine Analogs Exhibit Differential Protect...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,58,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","aminoglycoside, berbamine, hair cell, hearing ...",0
32848620,Tyrosine hydroxylase (Th) expression has previ...,Purkinje Cell-Specific Knockout of Tyrosine Hy...,Frontiers in cellular neuroscience,14,2020,"Sant'Ana Leandro de Oliveira, Machado Sérgio, ...",None,124,"Janeiro, Brazil., Janeiro, Brazil., Janeiro, B...","catecholamine, cerebellum, cognition, dopamine...",0


In [162]:
df_abstract = df[['abstract']]
df_abstract

,abstract
PMID,
35961771,Navigation through complex environments requir...
35961623,Recent evidence shows that genetic and environ...
35961621,This study aimed to examine the prospective as...
35961602,Psychosis presentation can be affected by gene...
35961582,Mental health and cognitive achievement are pa...
...,...
32848625,The inferior colliculus (IC) is an auditory mi...
32848624,Hearing loss is the third most common chronic ...
32848620,Tyrosine hydroxylase (Th) expression has previ...


In [163]:
df_title = df[['articleTitle']]
df_title

,articleTitle
PMID,
35961771,Coordination between eye movement and whisking...
35961623,Polygenic risk for schizophrenia as a moderato...
35961621,The association of substance use with attainin...
35961602,Differences of affective and non-affective psy...
35961582,Brain structure and function show distinct rel...
...,...
32848625,Developmentally Regulated Rebound Depolarizati...
32848624,Berbamine Analogs Exhibit Differential Protect...
32848620,Purkinje Cell-Specific Knockout of Tyrosine Hy...


In [164]:
df_title.dtypes

articleTitle    object
dtype: object

In [166]:
def clean_txt(text):
    
    text = text.lower()
    
    for punctuation in string.punctuation:
        
        text = text.replace(punctuation,'')
              
    text = ''.join(char for char in text if not char.isdigit()) 
    
    tokenized_text = word_tokenize(text)
    stop_words = set(stopwords.words('english')) 
    tokenized_sentence_cleaned = [w for w in tokenized_text 
                                  if not w in stop_words]
    
    verb_lemmatized = [WordNetLemmatizer().lemmatize(word, pos = "v")
              for word in tokenized_sentence_cleaned]
    
    noun_lemmatized = [WordNetLemmatizer().lemmatize(word, pos = "n")  # n --> nouns
               for word in verb_lemmatized]
        
    return " ".join(noun_lemmatized)

In [167]:
df_title.articleTitle = df_title.articleTitle.astype(str).apply(clean_txt)

df_title

,articleTitle
PMID,
35961771,coordination eye movement whisk head fix mouse...
35961623,polygenic risk schizophrenia moderator associa...
35961621,association substance use attain employment am...
35961602,difference affective nonaffective psychosis ea...
35961582,brain structure function show distinct relatio...
...,...
32848625,developmentally regulate rebound depolarizatio...
32848624,berbamine analog exhibit differential protecti...
32848620,purkinje cellspecific knockout tyrosine hydrox...


In [168]:
df_abstract.abstract = df_abstract.abstract.astype(str).apply(clean_txt)
df_abstract

,abstract
PMID,
35961771,navigation complex environment require motor p...
35961623,recent evidence show genetic environmental ris...
35961621,study aim examine prospective association toba...
35961602,psychosis presentation affect genetic environm...
35961582,mental health cognitive achievement partly her...
...,...
32848625,inferior colliculus ic auditory midbrain struc...
32848624,hear loss third common chronic health conditio...
32848620,tyrosine hydroxylase th expression previously ...


In [156]:
tfidf_vectorizer_abstract = TfidfVectorizer()
tfidf_vectorizer_title =TfidfVectorizer()

In [157]:
tfidf_abstract = tfidf_vectorizer_abstract.fit_transform(df_abstract.abstract)
tfidf_abstract

<88070x197778 sparse matrix of type '<class 'numpy.float64'>'
	with 8007116 stored elements in Compressed Sparse Row format>

In [158]:
tfidf_title = tfidf_vectorizer_title.fit_transform(df_title.articleTitle)
tfidf_title

<88070x50248 sparse matrix of type '<class 'numpy.float64'>'
	with 870832 stored elements in Compressed Sparse Row format>

In [214]:
search_terms_list_1 = ["brain imaging animal"]#"neuronal circuit","voltage imaging",
search_terms_list_2 = ["in vivo imaging","multiphoton","optogenetics","intravital","pre-clinical"]
search_terms_list_3 = ["patchclamp","neurodegenerative"]

input_topic = " ".join(search_terms_list_1)

In [186]:
input_topic

'neuronal circuit voltage imaging brain imaging animal mouse mice'

In [215]:
cleaned_input = pd.DataFrame({'Predict searc':clean_txt(input_topic)},index=[1])
cleaned_input

,Predict searc
1,brain image animal


In [216]:
input_tfidf_title = tfidf_vectorizer_title.transform(cleaned_input)
input_tfidf_title

<1x50248 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [218]:
input_tfidf_abstract = tfidf_vectorizer_abstract.transform(cleaned_input)
input_tfidf_abstract

<1x197778 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [223]:
n_neighbors = 100

KNN_t = NearestNeighbors(n_neighbors)
KNN_t.fit(tfidf_title)

NNs_t = KNN_t.kneighbors(input_tfidf_title,return_distance=True)

df.iloc[list(NNs_t[1][0]), :]

recommendations_by_title = df.iloc[list(NNs_t[1][0]), :]

recommendations_by_title.iloc[2]['articleTitle']

'Predicting changes in osmolality.'

In [221]:
n_neighbors = 100

KNN_a = NearestNeighbors(n_neighbors)
KNN_a.fit(tfidf_abstract)

NNs_a = KNN_a.kneighbors(input_tfidf_abstract,return_distance=True)

recommendations_by_abstract = df.iloc[list(NNs_a[1][0]), :]
recommendations_by_abstract.iloc[1]['abstract']

'To investigate the relationship between EEG activity and the global and domain specific cognitive performance of healthy nurses, and determine the predictive capabilities of these relationships. Sixty-four nurses were recruited for the present study, and data from 61 were utilised in the present analysis. Global and domain specific cognitive performance of each participant was assessed psychometrically using the Mini-mental state exam and the Cognistat, and a 32-lead monopolar EEG was recorded during a resting baseline phase and an active phase in which participants completed the Stroop test. Global cognitive performance was successfully predicted (81%-85% of variance) by a combination of fast wave activity variables in the alpha, beta and theta frequency bands. Interestingly, predicting domain specific performance had varying degrees of success (42%-99% of the variance predicted) and relied on combinations of both slow and fast wave activity, with delta and gamma activity predicting 